# Customer Lifetime Value Prediction based on their historical interactions.

In [26]:
!pip install openpyxl
!pip install lifetimes
import datetime as dt
import pandas as pd
import matplotlib.pyplot as plt
from lifetimes import BetaGeoFitter
from lifetimes import GammaGammaFitter
from lifetimes.plotting import plot_period_transactions
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 500)
pd.set_option('display.float_format', lambda x: '%.4f' % x)
from sklearn.preprocessing import MinMaxScaler

DEPRECATION: torchsde 0.2.5 has a non-standard dependency specifier numpy>=1.19.*; python_version >= "3.7". pip 23.3 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of torchsde or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063
DEPRECATION: torchsde 0.2.5 has a non-standard dependency specifier numpy>=1.19.*; python_version >= "3.7". pip 23.3 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of torchsde or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063


In [27]:
df_ = pd.read_excel("online_retail_II.xlsx", sheet_name="Year 2010-2011")
df = df_.copy()

In [28]:
def outlier_thresholds(dataframe, variable):
    quartile1 = dataframe[variable].quantile(0.01)
    quartile3 = dataframe[variable].quantile(0.99)
    interquantile_range = quartile3 - quartile1
    up_limit = quartile3 + 1.5 * interquantile_range
    low_limit = quartile1 - 1.5 * interquantile_range
    return low_limit, up_limit

def replace_with_thresholds(dataframe, variable):
    low_limit, up_limit = outlier_thresholds(dataframe, variable)
    dataframe.loc[(dataframe[variable] < low_limit), variable] = low_limit
    dataframe.loc[(dataframe[variable] > up_limit), variable] = up_limit
    
    
df = df[df["Country"].str.contains("United Kingdom", na=False)] # UK selection of customers
df.dropna(inplace=True)
df = df[~df["Invoice"].str.contains("C", na=False)] # we did not receive those that started with c on the invoice in our data set. # RETURNS
df = df[df["Quantity"] > 0] # We have selected at least 1 purchase
df = df[df["Price"] > 0] # We chose those who earned more than 0
replace_with_thresholds(df, "Quantity") #  we use suppression.
replace_with_thresholds(df, "Price")    #  we use suppression.
df["TotalPrice"] = df["Quantity"] * df["Price"] # Let's create a total price variable
df["InvoiceDate"].max() # our latest invoice date
today_date = dt.datetime(2011, 12, 11) # our analysis date

In [29]:
cltv_df = df.groupby('Customer ID').agg({'InvoiceDate': [lambda date: (date.max() - date.min()).days, 
                                                        # Recency calculation: The time since the last purchase. Weekly.
                                                         lambda date: (today_date - date.min()).days], 
                                                        # T: How long before the analysis date was made the first purchase. Weekly.
                                         'Invoice': lambda num: num.nunique(), 
                                        # frequency : total number of recurring purchases unique
                                         'TotalPrice': lambda TotalPrice: TotalPrice.sum()}) 
                                        # monetary : average earnings per purchase
cltv_df.columns = cltv_df.columns.droplevel(0) # we are correcting the hierarchical column structure
cltv_df.columns = ['recency', 'T', 'frequency', 'monetary']

cltv_df["monetary"] = cltv_df["monetary"] / cltv_df["frequency"]
cltv_df = cltv_df[(cltv_df['frequency'] > 1)]
cltv_df = cltv_df[cltv_df["monetary"] > 0]
cltv_df["recency"] = cltv_df["recency"] / 7 # transaction to receive weekly
cltv_df["T"] = cltv_df["T"] / 7


In [30]:
bgf = BetaGeoFitter(penalizer_coef=0.001)
bgf.fit(cltv_df['frequency'],
        cltv_df['recency'],
        cltv_df['T'])


# "expected_purc_1_week"
cltv_df["expected_purc_1_week"] = bgf.predict(1,
                                              cltv_df['frequency'],
                                              cltv_df['recency'],
                                              cltv_df['T'])


# "expected_purc_1_month"
cltv_df["expected_purc_1_month"] = bgf.predict(4,
                                               cltv_df['frequency'],
                                               cltv_df['recency'],
                                               cltv_df['T'])

In [31]:
ggf = GammaGammaFitter(penalizer_coef=0.01)
ggf.fit(cltv_df['frequency'], cltv_df['monetary'])

# "expected_average_profit"
cltv_df["expected_average_profit"] = ggf.conditional_expected_average_profit(cltv_df['frequency'],
                                                                             cltv_df['monetary'])

In [32]:
cltv = ggf.customer_lifetime_value(bgf,
                                   cltv_df['frequency'],
                                   cltv_df['recency'],
                                   cltv_df['T'],
                                   cltv_df['monetary'],
                                   time=6,  # 6 months.
                                   freq="W",  # T's frequency information.
                                   discount_rate=0.01)
cltv.head()

Customer ID
12747.0000    1937.0091
12748.0000   12366.0719
12749.0000    3445.9230
12820.0000     631.9341
12822.0000    1612.1330
Name: clv, dtype: float64

In [33]:
cltv = cltv.reset_index()
cltv.sort_values(by="clv", ascending=False).head()


,Customer ID,clv
2486,18102.0000,85648.4974
589,14096.0000,55646.8292
2184,17450.0000,48531.9529
2213,17511.0000,36796.0328
1804,16684.0000,25082.3503


In [34]:
cltv_final = cltv_df.merge(cltv, on="Customer ID", how="left")
cltv_final.sort_values(by="clv", ascending=False).head(10)

,Customer ID,recency,T,frequency,monetary,expected_purc_1_week,expected_purc_1_month,expected_average_profit,clv
2486,18102.0000,52.2857,52.5714,60,3584.8878,0.9653,3.8507,3595.1928,85648.4974
589,14096.0000,13.8571,14.5714,17,3159.0771,0.7230,2.8733,3191.3874,55646.8292
2184,17450.0000,51.2857,52.5714,46,2629.5299,0.7451,2.9722,2639.4196,48531.9529
2213,17511.0000,52.8571,53.4286,31,2921.9519,0.5074,2.0240,2938.2748,36796.0328
1804,16684.0000,50.4286,51.2857,28,2120.0470,0.4767,1.9014,2133.2039,25082.3503
406,13694.0000,52.7143,53.4286,50,1267.3626,0.7982,3.1844,1271.7855,25060.0017
587,14088.0000,44.5714,46.1429,13,3859.6015,0.2597,1.0357,3911.3200,25009.4575
1173,15311.0000,53.2857,53.4286,91,667.5968,1.4286,5.6992,668.8945,23590.6848
133,13089.0000,52.2857,52.8571,97,605.1866,1.5320,6.1115,606.2937,22927.0048
1057,15061.0000,52.5714,53.2857,48,1108.3078,0.7693,3.0689,1112.3472,21122.4873


In [35]:
scaler = MinMaxScaler(feature_range=(0, 1))
scaler.fit(cltv_final[["clv"]])
cltv_final["scaled_clv"] = scaler.transform(cltv_final[["clv"]])
cltv_final.sort_values(by="scaled_clv", ascending=False).head()

,Customer ID,recency,T,frequency,monetary,expected_purc_1_week,expected_purc_1_month,expected_average_profit,clv,scaled_clv
2486,18102.0000,52.2857,52.5714,60,3584.8878,0.9653,3.8507,3595.1928,85648.4974,1.0000
589,14096.0000,13.8571,14.5714,17,3159.0771,0.7230,2.8733,3191.3874,55646.8292,0.6497
2184,17450.0000,51.2857,52.5714,46,2629.5299,0.7451,2.9722,2639.4196,48531.9529,0.5666
2213,17511.0000,52.8571,53.4286,31,2921.9519,0.5074,2.0240,2938.2748,36796.0328,0.4296
1804,16684.0000,50.4286,51.2857,28,2120.0470,0.4767,1.9014,2133.2039,25082.3503,0.2929


In [36]:
cltv_1 = ggf.customer_lifetime_value(bgf,
                                   cltv_df['frequency'],
                                   cltv_df['recency'],
                                   cltv_df['T'],
                                   cltv_df['monetary'],
                                   time=1,  # 1 MONTH 
                                   freq="W",  # T's frequency information.
                                   discount_rate=0.01)
cltv_1.head()

Customer ID
12747.0000    336.7716
12748.0000   2148.4185
12749.0000    604.0537
12820.0000    110.1237
12822.0000    286.9272
Name: clv, dtype: float64

In [37]:
cltv_1 = cltv_1.reset_index()
cltv_1.sort_values(by="clv", ascending=False).head(10)

,Customer ID,clv
2486,18102.0000,14884.5005
589,14096.0000,9855.1423
2184,17450.0000,8434.5076
2213,17511.0000,6394.1394
1804,16684.0000,4360.9256
587,14088.0000,4355.3691
406,13694.0000,4354.3348
1173,15311.0000,4098.7367
133,13089.0000,3983.9218
1485,16000.0000,3843.4088


In [38]:
cltv_1_final = cltv_df.merge(cltv_1, on="Customer ID", how="left")
cltv_1_final.sort_values(by="clv", ascending=False).head(10)

,Customer ID,recency,T,frequency,monetary,expected_purc_1_week,expected_purc_1_month,expected_average_profit,clv
2486,18102.0000,52.2857,52.5714,60,3584.8878,0.9653,3.8507,3595.1928,14884.5005
589,14096.0000,13.8571,14.5714,17,3159.0771,0.7230,2.8733,3191.3874,9855.1423
2184,17450.0000,51.2857,52.5714,46,2629.5299,0.7451,2.9722,2639.4196,8434.5076
2213,17511.0000,52.8571,53.4286,31,2921.9519,0.5074,2.0240,2938.2748,6394.1394
1804,16684.0000,50.4286,51.2857,28,2120.0470,0.4767,1.9014,2133.2039,4360.9256
587,14088.0000,44.5714,46.1429,13,3859.6015,0.2597,1.0357,3911.3200,4355.3691
406,13694.0000,52.7143,53.4286,50,1267.3626,0.7982,3.1844,1271.7855,4354.3348
1173,15311.0000,53.2857,53.4286,91,667.5968,1.4286,5.6992,668.8945,4098.7367
133,13089.0000,52.2857,52.8571,97,605.1866,1.5320,6.1115,606.2937,3983.9218
1485,16000.0000,0.0000,0.4286,3,2055.7867,0.4159,1.6407,2181.3264,3843.4088


In [39]:
scaler = MinMaxScaler(feature_range=(0, 1))
scaler.fit(cltv_1_final[["clv"]])
cltv_1_final["scaled_clv_1ay"] = scaler.transform(cltv_1_final[["clv"]])
cltv_1_final.sort_values(by="scaled_clv_1ay", ascending=False).head(10)

,Customer ID,recency,T,frequency,monetary,expected_purc_1_week,expected_purc_1_month,expected_average_profit,clv,scaled_clv_1ay
2486,18102.0000,52.2857,52.5714,60,3584.8878,0.9653,3.8507,3595.1928,14884.5005,1.0000
589,14096.0000,13.8571,14.5714,17,3159.0771,0.7230,2.8733,3191.3874,9855.1423,0.6621
2184,17450.0000,51.2857,52.5714,46,2629.5299,0.7451,2.9722,2639.4196,8434.5076,0.5667
2213,17511.0000,52.8571,53.4286,31,2921.9519,0.5074,2.0240,2938.2748,6394.1394,0.4296
1804,16684.0000,50.4286,51.2857,28,2120.0470,0.4767,1.9014,2133.2039,4360.9256,0.2930
587,14088.0000,44.5714,46.1429,13,3859.6015,0.2597,1.0357,3911.3200,4355.3691,0.2926
406,13694.0000,52.7143,53.4286,50,1267.3626,0.7982,3.1844,1271.7855,4354.3348,0.2925
1173,15311.0000,53.2857,53.4286,91,667.5968,1.4286,5.6992,668.8945,4098.7367,0.2754
133,13089.0000,52.2857,52.8571,97,605.1866,1.5320,6.1115,606.2937,3983.9218,0.2677
1485,16000.0000,0.0000,0.4286,3,2055.7867,0.4159,1.6407,2181.3264,3843.4088,0.2582


In [40]:
cltv_12 = ggf.customer_lifetime_value(bgf,
                                   cltv_df['frequency'],
                                   cltv_df['recency'],
                                   cltv_df['T'],
                                   cltv_df['monetary'],
                                   time=12,  # 12 MONTH 
                                   freq="W",  # T's frequency information.
                                   discount_rate=0.01)
cltv_12.head()

Customer ID
12747.0000    3698.3185
12748.0000   23624.5767
12749.0000    6538.6808
12820.0000    1204.3188
12822.0000    3029.8755
Name: clv, dtype: float64

In [41]:
cltv_12 = cltv_12.reset_index()
cltv_12.sort_values(by="clv", ascending=False).head(10)

,Customer ID,clv
2486,18102.0000,163586.7180
589,14096.0000,104893.7419
2184,17450.0000,92691.9029
2213,17511.0000,70283.9549
1804,16684.0000,47889.1891
406,13694.0000,47870.6621
587,14088.0000,47687.8331
1173,15311.0000,45066.5705
133,13089.0000,43794.2635
1057,15061.0000,40347.7756


In [42]:
cltv_12_final = cltv_df.merge(cltv_12, on="Customer ID", how="left")
cltv_12_final.sort_values(by="clv", ascending=False).head(10)

,Customer ID,recency,T,frequency,monetary,expected_purc_1_week,expected_purc_1_month,expected_average_profit,clv
2486,18102.0000,52.2857,52.5714,60,3584.8878,0.9653,3.8507,3595.1928,163586.7180
589,14096.0000,13.8571,14.5714,17,3159.0771,0.7230,2.8733,3191.3874,104893.7419
2184,17450.0000,51.2857,52.5714,46,2629.5299,0.7451,2.9722,2639.4196,92691.9029
2213,17511.0000,52.8571,53.4286,31,2921.9519,0.5074,2.0240,2938.2748,70283.9549
1804,16684.0000,50.4286,51.2857,28,2120.0470,0.4767,1.9014,2133.2039,47889.1891
406,13694.0000,52.7143,53.4286,50,1267.3626,0.7982,3.1844,1271.7855,47870.6621
587,14088.0000,44.5714,46.1429,13,3859.6015,0.2597,1.0357,3911.3200,47687.8331
1173,15311.0000,53.2857,53.4286,91,667.5968,1.4286,5.6992,668.8945,45066.5705
133,13089.0000,52.2857,52.8571,97,605.1866,1.5320,6.1115,606.2937,43794.2635
1057,15061.0000,52.5714,53.2857,48,1108.3078,0.7693,3.0689,1112.3472,40347.7756


In [43]:
scaler = MinMaxScaler(feature_range=(0, 1))
scaler.fit(cltv_12_final[["clv"]])
cltv_12_final["scaled_clv_12ay"] = scaler.transform(cltv_12_final[["clv"]])
cltv_12_final.sort_values(by="scaled_clv_12ay", ascending=False).head(10)


,Customer ID,recency,T,frequency,monetary,expected_purc_1_week,expected_purc_1_month,expected_average_profit,clv,scaled_clv_12ay
2486,18102.0000,52.2857,52.5714,60,3584.8878,0.9653,3.8507,3595.1928,163586.7180,1.0000
589,14096.0000,13.8571,14.5714,17,3159.0771,0.7230,2.8733,3191.3874,104893.7419,0.6412
2184,17450.0000,51.2857,52.5714,46,2629.5299,0.7451,2.9722,2639.4196,92691.9029,0.5666
2213,17511.0000,52.8571,53.4286,31,2921.9519,0.5074,2.0240,2938.2748,70283.9549,0.4296
1804,16684.0000,50.4286,51.2857,28,2120.0470,0.4767,1.9014,2133.2039,47889.1891,0.2927
406,13694.0000,52.7143,53.4286,50,1267.3626,0.7982,3.1844,1271.7855,47870.6621,0.2926
587,14088.0000,44.5714,46.1429,13,3859.6015,0.2597,1.0357,3911.3200,47687.8331,0.2915
1173,15311.0000,53.2857,53.4286,91,667.5968,1.4286,5.6992,668.8945,45066.5705,0.2755
133,13089.0000,52.2857,52.8571,97,605.1866,1.5320,6.1115,606.2937,43794.2635,0.2677
1057,15061.0000,52.5714,53.2857,48,1108.3078,0.7693,3.0689,1112.3472,40347.7756,0.2466


In [44]:
# Creation of Segments by CLTV
cltv_final["segment"] = pd.qcut(cltv_final["scaled_clv"], 4, labels=["D", "C", "B", "A"])
cltv_final.head()

,Customer ID,recency,T,frequency,monetary,expected_purc_1_week,expected_purc_1_month,expected_average_profit,clv,scaled_clv,segment
0,12747.0000,52.2857,52.8571,11,381.4555,0.2025,0.8077,387.8230,1937.0091,0.0226,A
1,12748.0000,53.1429,53.4286,209,154.5642,3.2375,12.9159,154.7086,12366.0719,0.1444,A
2,12749.0000,29.8571,30.5714,5,814.4880,0.1671,0.6657,844.0953,3445.9230,0.0402,A
3,12820.0000,46.1429,46.7143,4,235.5850,0.1040,0.4146,247.0812,631.9341,0.0074,C
4,12822.0000,2.2857,12.5714,2,474.4400,0.1291,0.5127,520.8292,1612.1330,0.0188,B


In [45]:
cltv_final.sort_values(by="scaled_clv", ascending=False).head()


,Customer ID,recency,T,frequency,monetary,expected_purc_1_week,expected_purc_1_month,expected_average_profit,clv,scaled_clv,segment
2486,18102.0000,52.2857,52.5714,60,3584.8878,0.9653,3.8507,3595.1928,85648.4974,1.0000,A
589,14096.0000,13.8571,14.5714,17,3159.0771,0.7230,2.8733,3191.3874,55646.8292,0.6497,A
2184,17450.0000,51.2857,52.5714,46,2629.5299,0.7451,2.9722,2639.4196,48531.9529,0.5666,A
2213,17511.0000,52.8571,53.4286,31,2921.9519,0.5074,2.0240,2938.2748,36796.0328,0.4296,A
1804,16684.0000,50.4286,51.2857,28,2120.0470,0.4767,1.9014,2133.2039,25082.3503,0.2929,A


In [46]:
cltv_final.groupby("segment").agg({"mean","count", "sum"})


Customer ID                          recency                        T                    frequency               monetary                      expected_purc_1_week                 expected_purc_1_month                 expected_average_profit                        clv                        scaled_clv               
              count           sum       mean   count        sum    mean count        sum    mean     count   sum    mean    count         sum     mean                count      sum   mean                 count      sum   mean                   count         sum     mean count          sum      mean      count     sum   mean
segment                                                                                                                                                                                                                                                                                                                              
D               643 10098911.0000 15705.9269     643 14178.4286 22.0504   643 26039.4286 40.4968       643  1971  3.0653      643 114131.0746 177.4978                  643  45.6395 0.0710                   643 181.7732 0.2827                     643 123652.7309 192.3060   643  173276.9837  269.4821        643  2.0231 0.0031
C               642  9964904.0000 15521.6573     642 19807.7143 30.8531   642 24510.1429 38.1778       642  2563  3.9922      642 167358.4523 260.6829                  642  76.7351 0.1195                   642 305.5820 0.4760                     642 178512.9973 278.0576   642  456392.8282  710.8923        642  5.3287 0.0083
B               642 10012848.0000 15596.3364     642 19156.5714 29.8389   642 22545.1429 35.1170       642  3502  5.4548      642 225975.5615 351.9869                  642 103.6279 0.1614                   642 412.4717 0.6425                     642 237858.4165 370.4960   642  816194.3841 1271.3308        642  9.5296 0.0148
A               643  9896309.0000 15390.8383     643 20229.1429 31.4606   643 22195.8571 34.5192       643  7260 11.2908      643 376993.0591 586.3034                  643 175.1393 0.2724                   643 697.2859 1.0844                     643 391392.9787 608.6983   643 2447404.5255 3806.2279        643 28.5750 0.0444